In [3]:
import requests
import urllib3
import os
from dotenv import load_dotenv


In [16]:
# Define the path to the .env file
dotenv_path = os.path.join('PRIVATE', '.env')

# Read the .env file
file = open(dotenv_path, 'r')
lines = file.readlines()
file.close()

# Parse environment variables
env_vars = {}
for line in lines:
    key, value = line.strip().split('=')
    env_vars[key] = value

# Construct the payload using environment variables
payload = {
    'client_id': env_vars.get('CLIENT_ID'),
    'client_secret': env_vars.get('CLIENT_SECRET'),
    'refresh_token': env_vars.get('REFRESH_TOKEN'),
    'grant_type': "refresh_token",
    'f': 'json'
}

